In [1]:
%matplotlib inline
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
from os.path import join
from nilearn.masking import apply_mask
from nilearn.image import threshold_img,math_img,mean_img

#matfile = loadmat('abide_mars2_0050002_3726')
v1 = pd.read_csv(r'C:\expdata\ABIDE_2017_drz5000\Phenotypic_V1_0b_preprocessed1.csv')

y = v1.DX_GROUP.as_matrix()*-1+2

face_dir = r'C:\expdata\ABIDE_2017_drz5000\mergedata_S0323_230217_rsn10_drzstat'
abide_mask_dir = r'C:\expdata\ABIDE_2017_drz5000\ABIDE_drz5000_mask'


def getauc(X,y):
    fpr, tpr, thresholds = metrics.roc_curve(y, X)
    auc = metrics.auc(fpr, tpr) 
    return auc

thr = 0.95
for ii in range(10):
    print('RSN%d' % ii)

    #test = pd.read_csv('af3t1.csv')
    #data = test.as_matrix()
    #X = data[:, 1]
    #y = data[:,0] #取出TOF那一行
    facedata_ff = join(face_dir,'rsn%d' % ii,'allRmaps.nii.gz')
    abide_maskt1_ff =join(abide_mask_dir,'rsn%d' % ii,'twosample_tfce_corrp_tstat1.nii.gz')
    abide_maskt2_ff =join(abide_mask_dir,'rsn%d' % ii,'twosample_tfce_corrp_tstat2.nii.gz')


    try:
        maskmeant1 = np.mean(apply_mask(facedata_ff, math_img('img1>=%f' % thr,img1=abide_maskt1_ff)),1)
        #maskmeant1 = np.mean(apply_mask(facedata_ff, r'C:\expdata\ABIDE_2017_drz5000\a7t1mask_95.nii.gz'),1)
        auct1 = getauc(maskmeant1*-1,y)
        #maskmeant1 = np.mean(apply_mask(facedata_ff, math_img('img1>=%f' % thr,img1=abide_maskt1_ff)),1)
        #maskmeant1 = np.mean(apply_mask(facedata_ff, r'C:\expdata\ABIDE_2017_drz5000\a7t1mask_95.nii.gz'),1)
        auctx = getauc(maskmeant1*-1,y)
    except BaseException as e:
        print(e)

        maskmeant1 = y*0        
        auct1 = 0
    try:
        maskmeant2 = np.mean(apply_mask(facedata_ff, math_img('img1>=%f' % thr,img1=abide_maskt2_ff)),1)
        auct2 = getauc(maskmeant2,y)
    except:
        auct2 = 0
        maskmeant2 = y*0 

    if ii == 0:
        t1all = maskmeant1
        t2all = maskmeant2
    else:
        print("hihi2")
        t1all = np.vstack((t1all,maskmeant1))
        t2all = np.vstack((t2all,maskmeant2))

    print('thr:%.2f, t1:%.3f , t2:%.3f ' % (thr,auct1,auct2))


np.savetxt('tsall.csv',np.hstack((t1all.T,t2all.T)),fmt='%.5f', delimiter=',')



RSN0
The mask is invalid as it is empty: it masks all data.
thr:0.95, t1:0.000 , t2:0.640 
RSN1
The mask is invalid as it is empty: it masks all data.
hihi2
thr:0.95, t1:0.000 , t2:0.650 
RSN2
hihi2
thr:0.95, t1:0.615 , t2:0.654 
RSN3
hihi2
thr:0.95, t1:0.643 , t2:0.683 
RSN4
hihi2
thr:0.95, t1:0.630 , t2:0.627 
RSN5
hihi2
thr:0.95, t1:0.655 , t2:0.675 
RSN6
hihi2
thr:0.95, t1:0.664 , t2:0.678 
RSN7
hihi2
thr:0.95, t1:0.711 , t2:0.628 
RSN8
hihi2
thr:0.95, t1:0.697 , t2:0.682 
RSN9
hihi2
thr:0.95, t1:0.636 , t2:0.666 


In [16]:
import pandas as pd
import glob
import numpy as np
import shutil
#matfile = loadmat('abide_mars2_0050002_3726')
v1 = pd.read_csv(r'C:\expdata\ABIDE_2017_drz5000\Phenotypic_V1_0b_preprocessed1.csv')

y = v1.DX_GROUP.as_matrix()

In [ ]:
import os
print(os.path.isfile(abide_maskt1_ff))

In [ ]:
from nilearn import plotting,image
plotting.plot_stat_map(math_img('img1>=%f' % thr,img1=abide_maskt1_ff))

In [ ]:
plotting.plot_stat_map(image.index_img(facedata_ff, 4))

In [ ]:
x = apply_mask(facedata_ff, math_img('img1>=%f' % thr,img1=abide_maskt2_ff))

In [ ]:
x

In [ ]:
np.mean(x,1)

In [ ]:
from notebook.auth import passwd

In [ ]:
passwd()

In [ ]:
jupyter notebook --generate-config

In [ ]:
import matplotlib.pyplot as plt
from os.path import join
import numpy as np
from nilearn import plotting, input_data, image
import nibabel as nib
def get_connindex(rsn10_ff,maskrns10_ff, zthr=4):
    img_data= nib.load(rsn10_ff).get_data()
    maskrsn10 = nib.load(maskrns10_ff).get_data()
    connindex= np.zeros(10)
    temp = ((maskrsn10 > zthr) * img_data).reshape((-1,10))
    for ii in range(10):
        temp1= temp[:,ii]
        connindex[ii] =np.nanmean(temp1[np.nonzero(temp1)])
    print('Connindex:'+' '.join(['%.5f' % ii for ii in connindex.tolist()]))
    return connindex